In [1]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

In [2]:
#mask out seas-1620

ds_lu_1620 = xr.open_dataset('wrfout_d04_2011-06-23_11:00:00')
lu_1620 = ds_lu_1620['LU_INDEX']

lu_1620 = lu_1620[0,:,:]

del lu_1620.coords['XTIME']

lu_1620_df = lu_1620.to_dataframe().reset_index()

In [3]:
#mask out seas-2040s

ds_lu_2040s = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu_2040s = ds_lu_2040s['LU_INDEX']

lu_2040s = lu_2040s[0,:,:]

del lu_2040s.coords['XTIME']

lu_2040s_df = lu_2040s.to_dataframe().reset_index()

# 2010s

In [4]:
ds1 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymin.nc')
t2_1120_ssp245_dailymin = ds1['ssp245_t2_1120_era5_corrected_dailymin']

t2_1120_ssp245_dailymin_df = t2_1120_ssp245_dailymin.to_dataframe().reset_index()

#Use 28℃ as a threshold

t2_1120_dailymin_thres = (t2_1120_ssp245_dailymin_df[t2_1120_ssp245_dailymin_df['ssp245_t2_1120_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [5]:
#pick out nights with minimum temperature over 28℃
HNF_1120 = t2_1120_dailymin_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_1120_era5_corrected_dailymin'].count()
HNF_1120 = HNF_1120.rename(columns={'ssp245_t2_1120_era5_corrected_dailymin':'hot_nights_num'})

HNF_1120['hot_nights_num'] = HNF_1120['hot_nights_num']/10

In [6]:
#mask out the sea

HNF_1120 = pd.merge(HNF_1120,lu_1620_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HNF_1120['hot_nights_num'] = HNF_1120['hot_nights_num'].fillna(0)

HNF_1120.loc[HNF_1120.LU_INDEX == 16, 'hot_nights_num'] = 9999

HNF_1120 = HNF_1120.rename(columns={"hot_nights_num": "HNF"})

In [7]:
HNF_1120.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_1120.csv',index=False,columns=['HNF','XLAT','XLONG'])

HNF_1120.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_1120_withgrids.csv',index=False,columns=['XLAT','XLONG','HNF','south_north','west_east'])

# 2040s

In [8]:
ds1 = xr.open_dataset('ssp245_t2_4049_era5_corrected_dailymin.nc')
t2_4049_ssp245_dailymin = ds1['ssp245_t2_4049_era5_corrected_dailymin']

t2_4049_ssp245_dailymin_df = t2_4049_ssp245_dailymin.to_dataframe().reset_index()

#Use 28℃ as a threshold

t2_4049_dailymin_thres = (t2_4049_ssp245_dailymin_df[t2_4049_ssp245_dailymin_df['ssp245_t2_4049_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [9]:
#pick out nights with minimum temperature over 28℃
HNF_4049 = t2_4049_dailymin_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_4049_era5_corrected_dailymin'].count()
HNF_4049 = HNF_4049.rename(columns={'ssp245_t2_4049_era5_corrected_dailymin':'hot_nights_num'})

HNF_4049['hot_nights_num'] = HNF_4049['hot_nights_num']/10

In [10]:
#mask out the sea

HNF_4049 = pd.merge(HNF_4049,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HNF_4049['hot_nights_num'] = HNF_4049['hot_nights_num'].fillna(0)

HNF_4049.loc[HNF_4049.LU_INDEX == 16, 'hot_nights_num'] = 9999

HNF_4049 = HNF_4049.rename(columns={"hot_nights_num": "HNF"})

In [11]:
HNF_4049.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_4049.csv',index=False,columns=['HNF','XLAT','XLONG'])

HNF_4049.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_4049_withgrids.csv',index=False,columns=['XLAT','XLONG','HNF','south_north','west_east'])

# 2090s

In [12]:
ds1 = xr.open_dataset('ssp245_t2_9099_era5_corrected_dailymin.nc')
t2_9099_ssp245_dailymin = ds1['ssp245_t2_9099_era5_corrected_dailymin']

t2_9099_ssp245_dailymin_df = t2_9099_ssp245_dailymin.to_dataframe().reset_index()

#Use 28℃ as a threshold

t2_9099_dailymin_thres = (t2_9099_ssp245_dailymin_df[t2_9099_ssp245_dailymin_df['ssp245_t2_9099_era5_corrected_dailymin']>=28]).reset_index(drop=True)

In [13]:
#pick out nights with minimum temperature over 28℃
HNF_9099 = t2_9099_dailymin_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_9099_era5_corrected_dailymin'].count()
HNF_9099 = HNF_9099.rename(columns={'ssp245_t2_9099_era5_corrected_dailymin':'hot_nights_num'})

HNF_9099['hot_nights_num'] = HNF_9099['hot_nights_num']/10

In [14]:
#mask out the sea

HNF_9099 = pd.merge(HNF_9099,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HNF_9099['hot_nights_num'] = HNF_9099['hot_nights_num'].fillna(0)

HNF_9099.loc[HNF_9099.LU_INDEX == 16, 'hot_nights_num'] = 9999

HNF_9099 = HNF_9099.rename(columns={"hot_nights_num": "HNF"})

In [15]:
HNF_9099.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_9099.csv',index=False,columns=['HNF','XLAT','XLONG'])

HNF_9099.to_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_9099_withgrids.csv',index=False,columns=['XLAT','XLONG','HNF','south_north','west_east'])

# diff

In [17]:
HNF_1120 = pd.read_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_1120_withgrids.csv')

HNF_1120 = HNF_1120.replace(9999,np.nan)

HNF_1120 = HNF_1120.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [18]:
HNF_4049 = pd.read_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_4049_withgrids.csv')

HNF_4049 = HNF_4049.replace(9999,np.nan)

HNF_4049 = HNF_4049.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [19]:
HNF_9099 = pd.read_csv('/disk/r110/zzuoaa/paper_plots/plots_era5_correction_1120/script/fig4_hw_metrics/csv/HNF_9099_withgrids.csv')

HNF_9099 = HNF_9099.replace(9999,np.nan)

HNF_9099 = HNF_9099.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [20]:
#test

print((HNF_1120['west_east']-HNF_4049['west_east']).min())
print((HNF_1120['west_east']-HNF_4049['west_east']).max())
print((HNF_1120['south_north']-HNF_4049['south_north']).min())
print((HNF_1120['south_north']-HNF_4049['south_north']).max())
print((HNF_4049['west_east']-HNF_9099['west_east']).min())
print((HNF_4049['west_east']-HNF_9099['west_east']).max())
print((HNF_4049['south_north']-HNF_9099['south_north']).min())
print((HNF_4049['south_north']-HNF_9099['south_north']).max())

0
0
0
0
0
0
0
0


In [21]:
HNF_diff_2040s = HNF_1120.copy()
HNF_diff_2090s = HNF_1120.copy()

In [24]:
HNF_diff_2040s['diff'] = HNF_4049['HNF']-HNF_1120['HNF']

HNF_diff_2090s['diff'] = HNF_9099['HNF']-HNF_1120['HNF']

In [25]:
HNF_diff_2040s['diff'] = HNF_diff_2040s['diff'].fillna(9999)

HNF_diff_2090s['diff'] = HNF_diff_2090s['diff'].fillna(9999)

In [26]:
HNF_diff_2040s.to_csv('HNF_diff_2040s_withgrids.csv')
HNF_diff_2090s.to_csv('HNF_diff_2090s_withgrids.csv')